In [1]:
!pip install paho-mqtt ipywidgets

   ---------------------------------------- 0.0/67.2 kB ? eta -:--:--
   ---------------------------------------- 67.2/67.2 kB 1.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
import json
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display

# -----------------------------------------------------------------------------
# - 목적: 빠르게 포즈 보정/정지 명령을 MQTT로 쏴서 디버깅을 돕는 임시 리모컨
# - 주의: 운영 코드가 아니라 작업 편의용(테스트/개발용) 스크립트
# -----------------------------------------------------------------------------

# ===== 설정 =====
BROKER_IP    = "10.236.94.221"  # MQTT broker 
BROKER_PORT  = 1883             # MQTT 기본 포트
TOPIC_DEV    = "robot/agv1/dev" # dev 제어 토픽

# ===== MQTT 연결 =====
# client_id는 중복되면 브로커에서 기존 세션을 끊을 수 있으니 timestamp로 유니크하게 만든다.
client = mqtt.Client(client_id=f"cheat-controller-{int(time.time())}")

try:
    # 연결만 한다고 publish가 바로 안정적으로 도는 게 아니라,
    # loop_start()로 네트워크 루프를 돌려야 송수신/keepalive가 정상 동작한다.
    client.connect(BROKER_IP, BROKER_PORT, keepalive=60)
    client.loop_start()
    print(f"✅ 리모컨 연결 성공! ({BROKER_IP})")
except Exception as e:
    print(f"❌ 연결 실패: {e}")

# ===== 버튼 클릭 함수 =====
def send_sync(area_name):
    # 수신측(노드레드/로봇)이 아래 JSON 스키마를 인식한다는 전제
    # - cmd: fix_pose  (현재 pose를 특정 area로 강제 보정)
    # - area: CHARGE/WATER/... (UI에서 쓰는 영역 라벨)
    payload = {
        "cmd": "fix_pose",
        "area": area_name
    }

    # qos=0: 빠르고 가벼움(유실 가능).
    client.publish(TOPIC_DEV, json.dumps(payload), qos=0)

    # 로그 남긴다.
    print(f"🚀 [전송] 위치 강제 동기화 -> {area_name}")

# ===== UI 버튼 생성 =====
print("\n👇 로봇이 해당 위치를 지날 때 버튼을 누르세요 👇\n")

# 버튼 크기 통일(노트북 화면에서 한 번에 보기 좋게)
layout = widgets.Layout(width='120px', height='40px', margin='5px')

# NOTE: btn_style은 ipywidgets.Button의 style에 직접 적용하지 않으면 효과 없음.
btn_style = {'button_color': '#4CAF50', 'text_color': 'white'}  # 초록색

# UserApp 기준 영역 라벨
areas = ["CHARGE", "WATER", "DROP", "RES_A", "RES_B", "RES_C"]
btns = []

for area in areas:
    # 버튼 표시 텍스트는 area 라벨 그대로 사용
    btn = widgets.Button(description=area, layout=layout)

    # Python 클로저 특성상 for문 변수(area)가 람다에 "마지막 값"으로 잡힐 수 있다.
    # default arg로 area 값을 고정해서 각 버튼이 자기 area로 publish 하도록 보장.
    btn.on_click(lambda b, a=area: send_sync(a))

    btns.append(btn)

# 배치: 한 줄에 3개씩(모니터 기준 가독성)
row1 = widgets.HBox(btns[:3])
row2 = widgets.HBox(btns[3:])

display(row1, row2)

# ===== 비상 정지 버튼 =====
stop_btn = widgets.Button(
    description="🚨 긴급 정지",
    layout=widgets.Layout(width='380px', height='50px', margin='10px')
)
stop_btn.style.button_color = '#FF5252'  # 빨간색(눈에 띄게)

def send_stop(b):
    # task 채널에서 {"cmd":"stop"}을 받으면 즉시 정지 처리한다는 전제
    # qos=0: 여기서도 유실 가능. 필요하면 qos=1로 바꾸는 게 더 안전(다만 지연/중복 고려 필요).
    client.publish("robot/agv1/task", json.dumps({"cmd": "stop"}), qos=0)
    print("🛑 [전송] 긴급 정지 명령")

stop_btn.on_click(send_stop)
display(stop_btn)


C:\Users\wp3wk\AppData\Local\Temp\ipykernel_20996\3641788835.py:20: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=f"cheat-controller-{int(time.time())}")


❌ 연결 실패: timed out

👇 로봇이 해당 위치를 지날 때 버튼을 누르세요 👇



Button(description='🚨 긴급 정지', layout=Layout(height='50px', margin='10px', width='380px'), style=ButtonStyle(bu…